# Sentiment Analysis

Congrats, you finished the part on the data preparation and we can now move on to the more interesting part of using RNNs/LSTMs to process sequential data! But be careful, even if the previous notebook might seem to be a little bit boring, it is of great importance. We switched from images to text data in this exercise and remember the first steps that we did in our class were data related, too and they were essential for all the following exercises. So naturally, since we switched to text data in this exercie, make sure to have a good understanding of how the data has been prepared.

For the last I2DL exercise, we want to make use of Recurrent Neural Networks (RNNs) to process sequential data. We will stick with the same dataset that we have been looking at in the previous notebook, namely the [IMDb](https://ai.stanford.edu/~amaas/data/sentiment/) sentiment analysis dataset that contains positive and negative movie reviews.

<p class="aligncenter">
    <img src="images/IMDB.jpg" alt="centered image" />
</p>

Sentiment analysis is the task of predicting the sentiment of a text. In this notebook, you will train a network to process reviews from the dataset and evaluate whether it has been a positive or a negative review. Below are two examples:

<p class="aligncenter">
    <img src="attachment:Screenshot%20from%202021-01-25%2008-29-35.png" alt="centered image" />
</p>


# 0. Setup

As always, we first import some packages to setup the notebook.

In [1]:
!pip install pytorch-lightning==1.0.8 > /dev/null
!pip install torch==1.7.0+cu101 torchvision==0.8.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html > /dev/null
#!pip install torch==1.7.0+cpu torchvision==0.8.1+cpu -f https://download.pytorch.org/whl/torch_stable.html > /dev/null
!pip install torchtext==0.8

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.7.0+cu101 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.7.0+cu101 which is incompatible.
     |████████████████████████████████| 6.9 MB 9.8 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [12]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
sys.path.append('/content/gdrive/MyDrive/I2DL/exercise_11/')

Mounted at /content/gdrive/


In [13]:
import os
import random
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm_

from exercise_code.rnn.sentiment_dataset import (
    download_data,
    load_sentiment_data,
    load_vocab,
    SentimentDataset,
    collate
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Loading Data

As we have learned from the notebook 1, this time we not only load the raw data, but also have the corresponding vocabulary. Let us load the data that we prepared for you:

In [14]:
i2dl_exercises_path = os.path.dirname(os.path.abspath(os.getcwd()))
data_root = os.path.join(i2dl_exercises_path, "datasets", "SentimentData")
base_dir = download_data(data_root)
vocab = load_vocab(base_dir)
train_data, val_data, test_data = load_sentiment_data(base_dir, vocab)

print("number of training samples: {}".format(len(train_data)))
print("number of validation samples: {}".format(len(val_data)))
print("number of test samples: {}".format(len(test_data)))

number of training samples: 9154
number of validation samples: 3133
number of test samples: 3083


## Dataset Samples

Our raw data consists of tuples `(raw_text, token_list, token_indices, label)`. Let's sample some relatively short texts from our dataset to have a sense how it looks like:

In [15]:
sample_data0 = [datum for datum in train_data if len(datum[1]) < 20 and datum[-1] == 0] # negative
sample_data1 = [datum for datum in train_data if len(datum[1]) < 20 and datum[-1] == 1] # positive

# we sample 2 tuples each from positive set and negative set
sample_data = random.sample(sample_data0, 2) + random.sample(sample_data1, 2)
for text, tokens, indices, label in sample_data:
    print('Text: \n {}\n'.format(text))
    print('Tokens: \n {}\n'.format(tokens))
    print('Indices: \n {}\n'.format(indices))
    print('Label:\n {}\n'.format(label))
    print()

Text: 
 Comment this movie is impossible. Is terrible, very improbable, bad interpretation e direction. Not look!!!!!

Tokens: 
 ['comment', 'this', 'movie', 'is', 'impossible', 'is', 'terrible', 'very', 'improbable', 'bad', 'interpretation', 'e', 'direction', 'not', 'look']

Indices: 
 [626, 10, 13, 9, 1216, 9, 270, 41, 4724, 59, 2664, 1010, 397, 24, 164]

Label:
 0


Text: 
 no comment - stupid movie, acting average or worse... screenplay - no sense at all... SKIP IT!

Tokens: 
 ['no', 'comment', 'stupid', 'movie', 'acting', 'average', 'or', 'worse', 'screenplay', 'no', 'sense', 'at', 'all', 'skip', 'it']

Indices: 
 [62, 626, 303, 13, 76, 791, 51, 335, 880, 62, 325, 31, 30, 1338, 8]

Label:
 0


Text: 
 Adrian Pasdar is excellent is this film. He makes a fascinating woman.

Tokens: 
 ['adrian', 'pasdar', 'is', 'excellent', 'is', 'this', 'film', 'he', 'makes', 'a', 'fascinating', 'woman']

Indices: 
 [4712, 1, 9, 203, 9, 10, 19, 46, 169, 3, 1144, 332]

Label:
 1


Text: 
 Smallville 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

## Checking the Vocabulary

In the previous notebook, we discussed the need of a vocabulary for mapping words to unique integer IDs. Instead of creating the vocabulary manually, we provide you with the vocabulary. Let's have a look at some samples from the vocabulary of the dataset:

In [16]:
print('Vocabulary size:', len(vocab), '\n\n  Sample words\n{}'.format('-' * 20))
sample_words = random.sample(list(vocab.keys()), 10)
for word in sample_words:
    print(' {}'.format(word))

Vocabulary size: 5002 

  Sample words
--------------------
 bike
 understated
 midnight
 k
 reflect
 4th
 chuck
 mainly
 dull
 outing


Also we saw that there are already indices in the raw data that we loaded. We can check if the indices in the vocabulary match the raw data using the last sentence in `sample_data`. Words that are not in the vocabulary are assigned to the symbol `<unk>`. The output of the following cell should be the same as the indices in the last example of our loaded raw data:

In [17]:
# Last sample from above 
(text, tokens, indices, label) = sample_data[-1]
print('Text: \n {}\n'.format(text))
print('Tokens: \n {}\n'.format(tokens))
print('Indices: \n {}\n'.format(indices))
print('Label:\n {}\n'.format(label))

# Compare with the vocabulary
print('Indices drawn from vocabulary: \n {}\n'.format([vocab.get(x, vocab['<unk>']) for x in sample_data[-1][1]]))

Text: 
 Smallville episode Justice is the best episode of Smallville ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! It's my favorite episode of Smallville! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! !

Tokens: 
 ['smallville', 'episode', 'justice', 'is', 'the', 'best', 'episode', 'of', 'smallville', 'it', 's', 'my', 'favorite', 'episode', 'of', 'smallville']

Indices: 
 [1, 340, 1308, 9, 2, 91, 340, 5, 1, 8, 16, 47, 352, 340, 5, 1]

Label:
 1

Indices drawn from vocabulary: 
 [1, 340, 1308, 9, 2, 91, 340, 5, 1, 8, 16, 47, 35

## Wrapping to PyTorch Datasets

Great, the raw data is loaded properly and the vocabulary is matching. Let us wrap our data in a PyTorch dataset. For more details, check out the previous notebook and the corresponding dataset class defined in `exercise_code/rnn/sentiment_dataset.py`.

In [18]:
# Define a Dataset Class for train, val and test set
train_dataset = SentimentDataset(train_data)
val_dataset = SentimentDataset(val_data)
test_dataset = SentimentDataset(test_data)

# 2. Creating a Sentiment Classifier

After we have loaded the data, it is time to define a model and start training and testing.

## Evaluation Metrics

Since we just need to predict positive or negative, we can use `binary cross-entropy loss` to train our model. And accuracy can be used to assess our model's performance. We will use the following evaluation model to compute the accuracy.

In [19]:
bce_loss = nn.BCELoss()

@torch.no_grad()
def compute_accuracy(model, data_loader):
    corrects = 0
    total = 0
    device = next(model.parameters()).device
    
    for i, x in enumerate(data_loader):
        input = x['data'].to(device)
        lengths = x['lengths']
        label = x['label'].to(device)
        pred = model(input, lengths)
        corrects += ((pred > 0.5) == label).sum().item()
        total += label.numel()
        
        if i > 0  and i % 100 == 0:
            print('Step {} / {}'.format(i, len(data_loader)))
    
    return corrects / total

## Step 1: Design your own model

In this part, you need to create a classifier using the LSTM and Embedding layers you implemented in the previous section. For the LSTM, you may also use the PyTorch implementation.

![Screenshot%20from%202021-01-25%2008-43-02.png](attachment:Screenshot%20from%202021-01-25%2008-43-02.png)



<div class="alert alert-info">
    <h3>Task: Implement a Classifier</h3>
    
   Go to <code>exercise_code/rnn/text_classifiers.py</code> and implement the <code>RNNClassifier</code>. In the skeleton code, we inherited <code>nn.Module</code>. You can also inherit <code>LightningModule</code> if you want to use PyTorch Lightning.
</div>

This file is mostly empty but contains the expected class name, and the methods that your model needs to implement (only `forward()` basically). 
The only rules your model design has to follow are:
* Inherit from `torch.nn.Module` or `pytorch_lightning.LightningModule`
* Perform the forward pass in `forward()`.
* Have less than 2 million parameters
* Have a model size of less than 50MB after saving

After you finished, edit the below cell to make sure your implementation is correct. You should define the model yourself, which should be small enough (2 Mio. parameters) and have correct output format.

In [21]:
from exercise_code.rnn.tests import classifier_test, parameter_test
from exercise_code.rnn.text_classifiers import RNNClassifier

model = None

########################################################################
# TODO - Create a Model                                               #
########################################################################

model = RNNClassifier(len(vocab), 40, 30)

########################################################################
#                           END OF YOUR CODE                           #
########################################################################

# Check whether your model is sufficiently small and have a correct output format
parameter_test(model), classifier_test(model, len(vocab))

Total number of parameters: 208751
Your model is sufficiently small :)
All output tests are passed :)!


(True, True)

## Step 2: Train your own model

In this section, you need to train the classifier you created. Below, you can see some setup code we provided to you. Note the **collate function** used with the `DataLoader`. If you forgot why we need the collate function here, check this out in Notebook 1.

You are free to change the below configs (e.g. batch size, device setting etc.) as you wish.

In [32]:
# Training configs
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using {}...\n'.format(device))

# Move model to the device we are using
model = model.to(device)

# To tackle with the exploding gradient problem, you may want to set gclip and use clip_grad_norm_
# see the optional notebook for the explanation
gclip = None

# Dataloaders, note the collate function
train_loader = DataLoader(
  train_dataset, batch_size=16, collate_fn=collate, drop_last=True
)
val_loader = DataLoader(
  val_dataset, batch_size=16, collate_fn=collate, drop_last=False
)

Using cuda...



<div class="alert alert-info">
<h3>Task: Implement Training</h3>
    <p>
        In the below cell, you are expected to implement your training loop to train your model. You can use the training loader provided above for iterating over the data. If you want to evaluate your model periodically, you may use the validation loader provided above. You can use pure PyTorch or PyTorch Lightning.
   </p>
</div>

**Hints :**
* Use `torch.nn.BCELoss` as loss function
* Revise the previous exercises if you get stuck.


In [33]:
########################################################################
#                     TODO - Train Your Model                          #
########################################################################

optim = torch.optim.Adam(model.parameters())
epochs = 5
gclip = 30
for epoch in range(epochs):
  print('Epoch {}\n'.format(epoch))
  model.train()
  corrects = 0
  labels = 0
  total_loss = 0.0
  for i, data in enumerate(train_loader):
    seq=data['data'].to(device)
    label=data['label'].to(device)
    seq_lens=data['lengths']
    model.zero_grad()
    pred=model(seq, seq_lens)
    loss=bce_loss(pred, label)
    loss.backward()
    clip_grad_norm_(model.parameters(),max_norm=gclip)
    optim.step()
    corrects+=((pred>0.5)==label).sum().item()
    labels+=label.numel()
    total_loss+=loss.item()*label.numel()
  print('Loss: {}, AUC: {}'.format(total_loss/labels, corrects/labels))
  model.eval()
  print('Evaluation AUC:{} \n', compute_accuracy(model, val_loader))  

  ########################################################################
#                           END OF YOUR CODE                           #
########################################################################

Epoch 0

Loss: 0.12567178923652958, AUC: 0.9592438811188811
Step 100 / 196
Evaluation AUC:{} 
 0.8534950526651771
Epoch 1

Loss: 0.10861483752284642, AUC: 0.9621940559440559
Step 100 / 196
Evaluation AUC:{} 
 0.8534950526651771
Epoch 2

Loss: 0.08777712193119072, AUC: 0.9729020979020979
Step 100 / 196
Evaluation AUC:{} 
 0.8538142355569741
Epoch 3

Loss: 0.08491863765089505, AUC: 0.9723557692307693
Step 100 / 196
Evaluation AUC:{} 
 0.8538142355569741
Epoch 4

Loss: 0.07403749343002919, AUC: 0.9774912587412588
Step 100 / 196
Evaluation AUC:{} 
 0.8365783593999362


## Testing the Model

As you trained a model and improved it on the validation set, you can now test it on the test set.

In [34]:
test_loader = DataLoader(
  test_dataset, batch_size=8, collate_fn=collate, drop_last=False
)

print("accuracy on test set: {}".format(compute_accuracy(model, test_loader)))

Step 100 / 386
Step 200 / 386
Step 300 / 386
accuracy on test set: 0.8413882581900746


## Demo

Now that you trained a sufficiently good sentiment classifier, run the below cell and type some text to see some predictions (type exit to quit the demo). Since we used a small data, don't expect too much :).

In [35]:
from exercise_code.rnn.sentiment_dataset import tokenize

text = ''
w2i = vocab
while True:
    text = input()
    if text == 'exit':
        break

    words = torch.tensor([
        w2i.get(word, w2i['<unk>'])
        for word in tokenize(text)
    ]).long().to(device).view(-1, 1)  # T x B

    pred = model(words).item()
    sent = pred > 0.5
    
    print('Sentiment -> {}, Confidence -> {}'.format(
        ':)' if sent else ':(', pred if sent else 1 - pred
    ))
    print()

KeyboardInterrupt: ignored

# 3. Submission

If you got sufficient performance on the test data, you are ready to upload your model to the [server](https://i2dl.vc.in.tum.de/submission/) . As always, let's first save your final model.

In [36]:
from exercise_code.util.save_model import save_model

save_model(model, 'rnn_classifier.p')

...Your model is saved to models/rnn_classifier.p successfully!!


'models/rnn_classifier.p'

Congrats, you finished the last I2DL exercise! One last time this semester, let's prepare the submission:

In [37]:
# Now zip the folder for upload
from exercise_code.util.submit import submit_exercise

submit_exercise('exercise11')

relevant folders: ['models']
notebooks files: []
Adding folder models
Zipping successful! Zip is stored under: /content/exercise11.zip


# 4. Submission Instructions

Congratulations! You've just built your first image classifier! To complete the exercise, submit your final model to our submission portal - you probably know the procedure by now.

1. Go on [our submission page](https://i2dl.vc.in.tum.de/), register for an account and login. We use your matriculation number and send an email with the login details to the mail account associated. When in doubt, login into tum-online and check your mails there. You will get an id which we need in the next step.
2. Log into [our submission page](https://i2dl.vc.in.tum.de/) with your account details and upload the zip file.
3. Your submission will be evaluated by our system and you will get feedback about the performance of it. You will get an email with your score as well as a message if you have surpassed the threshold.
4. Within the working period, you can submit as many solutions as you want to get the best possible score.

# 5. Submission Goals

- Goal: Implement and train a recurrent neural network for sentiment analysis.
- Passing Criteria: Reach **Accuracy >= 83%** on __our__ test dataset. The submission system will show you your score after you submit.

- Submission start: __Thursday, January 13, 2022 - 13:00:00__
- Submission deadline: __Wednesday, January 19, 2022 - 15:59:59__ 
- You can make **$\infty$** submissions until the deadline. Your __best submission__ will be considered for bonus.